In [ ]:
import itertools
import networkx as nx
import numpy as np

import network

from preprocess_data import *
from datetime import date
from enum import Enum

import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from scipy import stats

In [ ]:
def regular_networks():
    for game, venue, (situation, pp) in itertools.product(
        GAMES,
        [Venue.HOME, Venue.AWAY],
        [(Situation.REGULAR, None)]
    ):
        res = build_networks(Game(game), venue=venue, situation=situation, pp=pp)
        time = calculate_time(Game(game), None)
        if res is not None:
            nw = network.process_graph_for_analysis(res["position_pass_network"], make_undirected=True)
            time = calculate_time(Game(game), pp)
            yield [game, venue.value, situation.value, len(nw.nodes), nx.is_connected(nw),
                  sum(w for (_, _, w) in nw.edges.data("n_passes")), time]

In [ ]:
def power_play_networks():
    for game, venue, pp in itertools.product(
        GAMES,
        [Venue.HOME, Venue.AWAY],
        list(map(PowerPlay, range(1, MAX_PENALTY_NUMBER + 1)))
    ):
        res = build_networks(Game(game), venue=venue, situation=Situation.POWER_PLAY, pp=pp)
        if res is not None:
            nw = network.process_graph_for_analysis(res["position_pass_network"], make_undirected=True)
            time = calculate_time(Game(game), [pp])
            if len(nw.nodes) > 0:
                yield [game, venue.value, f"PP {pp.penalty_no}", len(nw.nodes), nx.is_connected(nw), 
                       sum(w for (_, _, w) in nw.edges.data("n_passes")), time]

In [ ]:
def penalty_kill_networks():
    for game, venue, pp in itertools.product(
        GAMES,
        [Venue.HOME, Venue.AWAY],
        list(map(PowerPlay, range(1, MAX_PENALTY_NUMBER + 1)))
    ):
        res = build_networks(Game(game), venue=venue, situation=Situation.PENALTY_KILL, pp=pp)
        if res is not None:
            nw = network.process_graph_for_analysis(res["position_pass_network"], make_undirected=True)
            time = calculate_time(Game(game), [pp])
            if len(nw.nodes) > 0:
                yield [game, venue.value, f"PK {pp.penalty_no}", len(nw.nodes), nx.is_connected(nw), 
                       sum(w for (_, _, w) in nw.edges.data("n_passes")), time]

In [ ]:
regular_network_stats = pd.DataFrame(list(regular_networks()), columns = ["Game", "Venue", "Situation", "#Nodes", "Connected-P", "#Passes", "Time"])
power_play_network_stats = pd.DataFrame(list(power_play_networks()), columns = ["Game", "Venue", "Situation", "#Nodes", "Connected-P", "#Passes", "Time"])
penalty_kill_network_stats = pd.DataFrame(list(penalty_kill_networks()), columns = ["Game", "Venue", "Situation", "#Nodes", "Connected-P", "#Passes", "Time"])

In [ ]:
regular_network_stats

In [ ]:
power_play_network_stats

In [ ]:
penalty_kill_network_stats